In [6]:
import pandas as pd
from customrec_engine import engines_list, engines, RecommendationAbstract
import pprint
from sklearn.model_selection import train_test_split
from typing import List, Tuple

product_datas = [{
    "data_context": "books",
    "product_filepath": "data/products_books_v1_10_10.csv",
    "transactions_filepath": "data/transactions_books_v1_10_10.csv",
    "features": ["product_title", "product_image", "product_soup", "product_images"],
    "version": "1.0",
    "unique_name": "_books_v1_10_10",
}]


In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

results = {} # {data_context: [{strategy_name, accuracy, precision, recall, f1_score, are_roc}]}

for product_data in product_datas:
    pprint.pprint(product_data)
    print("looking at", "../" + product_data["product_filepath"])

    productdf =  pd.read_csv("../" + product_data["product_filepath"])
    transactiondf = pd.read_csv("../" + product_data["transactions_filepath"])
    
    training_df_arr = []
    
    
    # join transactions by same user_id. into a dict of user_id: [transactions]
    user_transactions = {}
    expected_hits = []
    hits = []
    for row in transactiondf[:100].iterrows():
        training_df_arr.append(row[1])
        user_id = row[1]["user_id"]
        if user_id not in user_transactions:
            user_transactions[user_id] = []
        user_transactions[user_id].append(row[1]['product_id'])
        expected_hits.append(1)
    
    # create df from transactionsdf
    transactiondf = pd.DataFrame(training_df_arr)
    
    # print("user_transactions", len(user_transactions))
    # print("transactiondf", transactiondf.shape)
    
    # print(user_transactions.values())
    
    # split trainning and testing transactions
    past_transactions, test_transactions = train_test_split(list(user_transactions.values()), test_size=.2, random_state=42)
    # Display the shapes of the resulting DataFrames
    # print("Training set shape:", len(past_transactions))
    # print("Testing set shape:", len(test_transactions))
    
    # for each engine rec. Train, test:
    for rec_engine_class in engines_list[:1]:
        print(rec_engine_class.strategy_name)
        rec_engine: RecommendationAbstract  = rec_engine_class(products=productdf, product_data=product_data, transactions = transactiondf)
        rec_engine.train(auto_save=False)
        
        for user_transactions in test_transactions:
            # Start test setup.
            # split in 3/4 if more than 4 transactions, or 1/2 if less than 4 transactions
            past_transactions, pred_transactions = train_test_split(user_transactions, test_size=.25, random_state=42)
            # print('========= ', len(user_transactions), '=========', len(past_transactions), len(pred_transactions))
            # print('attempting past_transactions', past_transactions)
            recs: List[Tuple[dict, float]] = rec_engine.recommend_from_past(past_transactions)
            # get the recommendations as a list of product_ids
            recommendation_ids = [rec[0]['product_id'] for rec in recs]
            print('recommendation_ids', recommendation_ids)
            # If any of the recommendations are in the pred_transactions, then it's a hit
            hit = 0
            for rec in recommendation_ids:
                if rec in pred_transactions:
                    hit = 1
                    break
            hits.append(hit)
    
    # Measure the performance of the recommendation engine
    accuracy = accuracy_score(expected_hits, hits)
    precision = precision_score(expected_hits, hits)
    recall = recall_score(expected_hits, hits)
    f1 = f1_score(expected_hits, hits)
    
    results[product_data["data_context"]] = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }
    
print(results)
            
        
    
    
    


{'data_context': 'books',
 'features': ['product_title',
              'product_image',
              'product_soup',
              'product_images'],
 'product_filepath': 'data/products_books_v1_10_10.csv',
 'transactions_filepath': 'data/transactions_books_v1_10_10.csv',
 'unique_name': '_books_v1_10_10',
 'version': '1.0'}
looking at ../data/products_books_v1_10_10.csv
WordVec
id_to_products length 15443
recommendation_ids ['0451526341', '051513628X', '0151008116', '0452280621', '0156006529', '0425145638', '0385484518', '0316899984', '0446356832', '0156005891']
recommendation_ids ['0553582127', '0805063897', '1558745157', '0380813815', '0767900383', '0425169863', '1878424319', '0440998050', '0312966970', '0671888587']


ValueError: Found input variables with inconsistent numbers of samples: [100, 2]